In [1]:
from sklearn.metrics import classification_report
import pandas as pd
import requests
import json
from tqdm import tqdm
import numpy as np
import time
import math
tqdm.pandas()

In [2]:
def json_read(path: str):
    with open(path, encoding="utf-8") as fh:
        data = json.load(fh)
    return data

In [3]:
labels = pd.read_csv("../csv/20230914_091933.csv")
filenames = labels["File Name"].unique().tolist()

In [5]:
call_data = pd.read_json("../json/data/dataM3_373call.json")
call_data = call_data[call_data["fileName"].isin(filenames)]
# call_data.sample()
call_data

,fileName,content,metadata
0,1691368375053_3752_810347419931_529288950.mp3,"[{'channel': 2, 'start': 3.469999999999999, 'e...",{'public': {'parent': 'metadata_20230807_16915...
1,1691368375053_3752_810347419931_529288950.mp3,"[{'channel': 2, 'start': 3.469999999999999, 'e...",{'public': {'parent': 'metadata_20230807_16915...
2,1691368375053_3752_810347419931_529288950.mp3,"[{'channel': 2, 'start': 3.469999999999999, 'e...",{'public': {'parent': 'metadata_20230807_16915...
3,1691368375053_3752_810347419931_529288950.mp3,"[{'channel': 2, 'start': 3.469999999999999, 'e...",{'public': {'parent': 'metadata_20230807_16915...
4,1691368375053_3752_810347419931_529288950.mp3,"[{'channel': 2, 'start': 3.469999999999999, 'e...",{'public': {'parent': 'metadata_20230807_16915...
...,...,...,...
6727,1694248856031_3736_810357789576_539270001.mp3,"[{'agent': True, 'channel': 1, 'start': 1.6800...","{'public': {'call_domain': 'M3', 'contract_id'..."
6728,1694248856031_3736_810357789576_539270001.mp3,"[{'agent': True, 'channel': 1, 'start': 1.6800...","{'public': {'call_domain': 'M3', 'contract_id'..."
6729,1694248856031_3736_810357789576_539270001.mp3,"[{'agent': True, 'channel': 1, 'start': 1.6800...","{'public': {'call_domain': 'M3', 'contract_id'..."
6730,1694248856031_3736_810357789576_539270001.mp3,"[{'agent': True, 'channel': 1, 'start': 1.6800...","{'public': {'call_domain': 'M3', 'contract_id'..."


In [6]:
m3_project_info = json_read("../json/project_info/m3_project_info_dev.json")["project_info"]
criteria_name = ["willpaySummary", "nopaySummary", "paidSummary"]

In [20]:
def test_criteria(project_info, criteria_name, transcript, metadata, agent_channel, file_name="test"):
    url = "https://cqe-dev-nlp.fpt.ai/dme/predict/dialogue/test"
    payload = json.dumps({
      "project_info": project_info,
      "transcript": transcript,
      "metadata": metadata,
      "criteria": criteria_name,
      "fileName": file_name,
      "agentChannel": agent_channel
    })
    headers = {
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

def calculate_criteira(rec, wc_project_info, criteria_name):
    while True:
        try:
            result = test_criteria(wc_project_info, criteria_name,
                           rec["content"], rec["metadata"],
                           agent_channel=1, file_name=rec["fileName"])
            for criteria in criteria_name:
                rec[criteria] = result[criteria]["decision"]
            break
        except:
            print(result.keys())
            print(rec["name"])
            time.sleep(10)
    return rec

In [21]:
sample = call_data.sample().to_dict("records")[0]
result = test_criteria(m3_project_info, criteria_name, sample["content"], sample["metadata"], agent_channel=1, file_name=sample["fileName"])
# sample

In [22]:
call_data = call_data.progress_apply(lambda rec: calculate_criteira(rec, m3_project_info, criteria_name), axis=1)

  0%|          | 25/6732 [00:41<3:07:10,  1.67s/it]


UnboundLocalError: cannot access local variable 'result' where it is not associated with a value

In [ ]:
call_data

,name,agentChannel,content,metadata,willpaySummary,nopaySummary,paidSummary
0,1685157935457_3738_850961387032_517774092.mp3,1,"[{'channel': 1, 'text': 'cái gì xin chào chào ...","{'parent': 'metadata_20230527_1685945582075', ...",yes,no,no
1,1685152174765_3764_810989107939_517731851.mp3,1,"[{'channel': 1, 'text': 'xin chào bên em gọi q...","{'parent': 'metadata_20230527_1685945582075', ...",no,no,no
2,1685158598464_3738_850972579342_522808569.mp3,1,"[{'channel': 1, 'text': 'dạ xin chào bên home ...","{'parent': 'metadata_20230527_1685945582075', ...",no,no,no
3,1685581769516_3600_850988632007_518568283.mp3,1,"[{'channel': 1, 'text': 'alo chào em bên home ...","{'parent': 'metadata_20230601_1685945569381', ...",no,no,no
4,1685851451096_3732_810912509169_519117581.mp3,1,"[{'channel': 1, 'text': 'chào phương home cred...","{'parent': 'metadata_20230604_1686546696463', ...",no,no,no
...,...,...,...,...,...,...,...
402,1685767446133_3466_810906924291_518985855.mp3,1,"[{'channel': 1, 'text': 'dạ alo chào nay mặt t...","{'parent': 'metadata_20230603_1686546710164', ...",no,no,no
407,1685927864453_3729_810974579084_519201989.mp3,1,"[{'channel': 1, 'text': 'chào linh gọi từ home...","{'parent': 'metadata_20230605_1686546722121', ...",no,no,no
411,1684981476107_3771_850966858098_522434696.mp3,1,"[{'channel': 2, 'text': 'alo'}, {'channel': 1,...","{'parent': 'metadata_20230525_1685360595626', ...",no,no,no
412,1685064372290_3743_810931516538_517542970.mp3,1,"[{'channel': 1, 'text': 'chào thi home credit ...","{'parent': 'metadata_20230526_1685360610102', ...",yes,no,no


In [ ]:
call_data.to_csv("../csv/callResultAI.csv", index=False, encoding="utf-8-sig")

In [ ]:
compare = labels.loc[labels["Criterion Name"] == "Contain all information"].merge(call_data, left_on='File Name', right_on='name')
compare = compare[["name", "Criterion Name", "Expected Result"] + criteria_name]
compare['Expected Result'] = compare['Expected Result'].replace(np.nan, "no")
compare["Expected Result"] = compare["Expected Result"].apply(str.lower)

In [ ]:
compare

,name,Criterion Name,Expected Result,willpaySummary,nopaySummary,paidSummary
0,1684482083958_3770_810983451276_516462765.mp3,Contain all information,no,no,no,no
1,1684482109334_3771_810932104194_516462776.mp3,Contain all information,no,no,no,no
2,1684484668825_3776_810365496726_516470077.mp3,Contain all information,no,no,no,no
3,1684484804994_3753_810989015916_516470478.mp3,Contain all information,no,no,no,no
4,1684485272349_3648_810939424196_516471469.mp3,Contain all information,no,no,no,no
...,...,...,...,...,...,...
295,1686275888826_3729_810973322412_519911491.mp3,Contain all information,yes,no,yes,no
296,1686276029433_3734_810981456551_519912816.mp3,Contain all information,yes,yes,no,no
297,1686279460634_3732_810909757150_519942265.mp3,Contain all information,no,no,no,no
298,1686285301787_3769_810337445113_519982400.mp3,Contain all information,no,no,no,no


In [ ]:
compare['Expected Result'] = compare['Expected Result'].replace("partially", "no")

In [ ]:
compare['Expected Result'].value_counts()

Expected Result
no     222
yes     78
Name: count, dtype: int64

In [ ]:
acc_willpay = 1-len(compare["Expected Result"].compare(compare["willpaySummary"]))/len(compare)
acc_nopay = 1-len(compare["Expected Result"].compare(compare["nopaySummary"]))/len(compare)
acc_paid = 1-len(compare["Expected Result"].compare(compare["paidSummary"]))/len(compare)

In [ ]:
print(acc_willpay)
print(acc_nopay)
print(acc_paid)

0.6866666666666666
0.6933333333333334
0.7333333333333334


In [ ]:
accuracy = np.mean([acc_willpay, acc_nopay, acc_paid])
print(f"Accuracy: {accuracy}")

Accuracy: 0.7044444444444444


In [ ]:
print(len(compare[(compare["Expected Result"]=="no") & (compare["willpaySummary"]=="yes")]))

70


In [ ]:
print(len(compare[(compare["Expected Result"]=="yes") & (compare["willpaySummary"]=="no")]))

24


In [ ]:
import json

with open('../json/data/m3_call_clean.json', 'r', encoding="utf-8") as file:
    # Tạo đối tượng csv reader
    reader =json.load(file)
    
result = []

for call in reader:
    file_name = call['name']
    agentChannel = call["agentChannel"]
    content = call['content']
    tr = []
    for sentences in content:
        text = {}
        text["channel"] = sentences['channel']
        text['text'] = sentences['text']
        tr.append(text)
    result.append({"file_name": file_name, "agentChannel": agentChannel, "content": tr})
    
with open("../json/content/result_content_m3.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [ ]:
len(result)

418

In [ ]:
y_true = [i for i in compare['Expected Result']]
y_pred_willpay = [i for i in compare['willpaySummary']]

In [ ]:
labels = ['yes', 'no']

In [ ]:
print("willpaySummary:")
print(classification_report(y_true, y_pred_willpay, labels=labels, zero_division=1))

willpaySummary:
              precision    recall  f1-score   support

         yes       0.44      0.69      0.53        78
          no       0.86      0.68      0.76       222

    accuracy                           0.69       300
   macro avg       0.65      0.69      0.65       300
weighted avg       0.75      0.69      0.70       300

